In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# 데이터 전처리
import re

# keybert
from keybert import KeyBERT
from transformers import BertModel, BertTokenizer

# 형태소 분석
from konlpy.tag import Okt

# 네트워크 분석
import networkx as nx

# DTM 만들기
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\wodnj\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
from keybert import KeyBERT
from transformers import BertModel, BertTokenizer

# kobert를 임베딩 모델로 사용
model = BertModel.from_pretrained('monologg/kobert')
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
kw_model = KeyBERT(model=model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [3]:
# data import
dg_festival = pd.read_csv('블로그_대구치맥페스티벌.csv')
dg_market = pd.read_csv('블로그_대구서문시장.csv')
ys_festival = pd.read_csv('블로그_예산맥주페스티벌.csv')
ys_market = pd.read_csv('블로그_예산시장.csv')

In [4]:
combined_Ydata = pd.concat([ys_market, ys_festival], axis=0, ignore_index=True)
combined_Ddata = pd.concat([dg_market, dg_festival], axis=0, ignore_index=True)

In [5]:
combined_Ydata

,title,author,datetime,contents
0,예산 가볼 만한 곳 백종원 시장이 되다 예산시장 다녀왔어요,이맛나,2023. 9. 14. 12:24,ː국내여행\n예산 가볼 만한 곳 백종원 시장이 되다 예산시장 다녀왔어요\n이맛나 ・...
1,예산시장 장터광장 카스테라 낙원약과 사과당 신양튀김,miss엉뚱,2023. 9. 8. 18:34,[블챌] 체크인 챌린지\n예산시장 장터광장 카스테라 낙원약과 사과당 신양튀김\nmi...
2,"예산시장 방문후기2-사과당, 신양튀김, 백술상회, 금오바베큐, 아가떡방(feat.맛...",미니미니쩜,2023. 9. 24. 5:16,"어디를 갈까?\n예산시장 방문후기2-사과당, 신양튀김, 백술상회, 금오바베큐, 아가..."
3,예산시장 '백종원 시장이 되다' 방문기,인라이너,2023. 8. 25. 17:15,2023 덕산여행\n예산시장 '백종원 시장이 되다' 방문기\n인라이너 ・ 2023....
4,백종원 회장님 예산 시장 구경 예산 시장 먹거리 살리기 맞나?,공쟁이,2023. 9. 22. 8:00,충청도여행\n백종원 회장님 예산 시장 구경 예산 시장 먹거리 살리기 맞나?\n공쟁이...
...,...,...,...,...
1974,푸드투데이 오늘의 식품 경제 생활 이슈,푸드투데이,2023. 8. 22. 18:06,푸드투데이\n푸드투데이 오늘의 식품 경제 생활 이슈\n푸드투데이 ・ 2023. 8....
1975,티켓예매는 OK캐쉬백으로,ssonglove1007,2023. 9. 14. 22:46,일상이야기\n티켓예매는 OK캐쉬백으로\nssonglove1007 ・ 2023. 9....
1976,[08.22. (화) 신문클리핑] 이병철 기자의 뉴스를 읽다,공정뉴스,2023. 8. 22. 8:52,공정뉴스\n[08.22. (화) 신문클리핑] 이병철 기자의 뉴스를 읽다\n공정뉴스 ...
1977,푸드투데이 오늘의 식품 경제 연예 이슈,푸드투데이,2023. 9. 1. 18:05,푸드투데이\n푸드투데이 오늘의 식품 경제 연예 이슈\n푸드투데이 ・ 2023. 9....


In [7]:
# NaN 값을 빈 문자열로 대체
combined_Ydata['contents'] = combined_Ydata['contents'].fillna('')

keywords_list = []

# 키워드 추출
for i in combined_Ydata['contents']:
    # i를 문자열로 변환하여 안전하게 처리
    keywords = kw_model.extract_keywords(str(i), keyphrase_ngram_range=(1, 1), use_mmr=True, diversity=0.5)
    keywords_list.append(keywords)

# 수치만 제외하고 단어만 추출해서 리스트 안에 묶기
strings_Yjoined = [' '.join([word for word, _ in sublist]) for sublist in keywords_list]

# 결과 출력
strings_Yjoined

['어제 진영상회 유튜브에 mbc 예산시장',
 '예산시장에 바뀐 주차장 관광지라고 11시에',
 '발라줘서ㅋㅋㅋ 기술력인가 972 예산시장메뉴추천 무책임',
 '주차장도 예산시장보다 가게들이 저렴하다 살펴보니',
 '예산군 967 의자가 50m 지붕',
 '예산에 며칠 50m 중국집은 url',
 '예산시장에서 착한금액 너무너무 이웃추가 유튜브에서',
 '태평소국밥에 record 60년전통예산장터국밥 50m 예산시장도',
 '몰려있어서ㅠㅠ 유튜브를 예산시장 url 50m',
 '적은듯해요ㅋㅋㅋ 32 url 예산시장 메뉴',
 '느린여행 생활관광지로 즐겨보시고요 걷는 예산형',
 '카페도 저렴해서 예산 토요일까지 기온도',
 '예산시장_예산맥주페스티벌 예산맥주패스티벌 맥주맛 다음날 예산시장',
 '충남 장터광장의 읍내사거리 예산시장 600g',
 '끊어지게 주차장 예산맥주페스티벌셔틀버스시간표 450g 예산시장선봉국수',
 'ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ com 시장활성화를 url 의자가',
 '예산시장을 오픈했다가 서울에도 재점검후 041',
 '예산시장에 제멋대로라 못했어요 50m url',
 '예산시장의 이유 건물주들이 mbc 137만',
 '예산시장에서 식당들이 10시까지 광장입니다 tour',
 '예산군 서울에서 유튜브에서 의좋은형제공원 메뉴',
 '추천ㅠㅠㅠㅠ흐규규귭 핵이득일 30m url 예산시장',
 '예산종합터미널까지 서울에서 mbc에서 602번 url',
 '더느린여행 백종원예산장터광장에서 느린여행 64m 예산시장',
 '예산군 주말에는 광장 맛집 덥습니다',
 '더본코리아 개발을 4월에 예산상설시장 50m',
 '김말이처럼 13 예산시장 url 마지막날',
 '김말이와 충남예산시장 이용방법 gate 70년',
 '예산 경기농업현장학습 40년 캐나다에서 비가',
 '예산시장이 방문객들이 미래 1월 2년',
 '충남도민리포터 국밥 유튜브 예약 예산시장',
 '주말이라 없었지만 비가 진한멸치국수 예산시장',
 '처음이쥬 041 50m url 

In [15]:
# 키워드 추출 데이터 csv 저장
import csv

# 리스트를 CSV 파일로 저장
with open('strings_Yjoined_blog.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for item in strings_Yjoined:
        writer.writerow([item])  # 각 리스트 요소를 한 행에 저장

In [11]:
combined_Ddata['contents'] = combined_Ddata['contents'].fillna('')

keywords_list = []

# 키워드 추출
for i in combined_Ddata['contents']:
    # i를 문자열로 변환하여 안전하게 처리
    keywords = kw_model.extract_keywords(str(i), keyphrase_ngram_range=(1, 1), use_mmr=True, diversity=0.5)
    keywords_list.append(keywords)

# 수치만 제외하고 단어만 추출해서 리스트 안에 묶기
strings_Djoined = [' '.join([word for word, _ in sublist]) for sublist in keywords_list]


# 결과 출력
strings_Djoined

['생방송투데이 김밥에는 놀라운토요일 영업시간 url',
 '김민경님이 visitkoreayear2023 6시 256 url',
 '소개해드릴게요 관광명소 소문시장 url 2016년부터는',
 '투데이와 365일 053 url 서문시장',
 '포토존으로 url 서문시장 50m 2016년',
 '변동있다고하니 0507 시장 50m url',
 '한국그릇백화점에서 추천해달라고 저렴 50m url',
 '재래시장은 10분초과시 50 url 119안전센터가',
 '대구서문시장맛집 12시 맛있어서 url 가격대가',
 '비산동디저트맛집 2023 url 앉아 50m',
 '서문시장갈비찜맛집 시장 url 50m 폭풍',
 '재래시장을 영업시간 저렴하다 url 50m',
 '좋아할법한 11시30분 무료로 운영기간 url',
 '오늘소개할곳은 223133930515 url 서문돈가스 50m',
 'ㅋㅋ같이데려가준 2023 임신한 url 50m',
 '버리고ㅠㅠㅠ아까버 32 졸업 매장사진 url',
 '바닐라라떼는 2023 url 디카페인 의자',
 '결혼준비 웨딩홀 5개월 10년 출구로',
 '갈비찜집이 10분당 256 url 시장특성상',
 '카페인데요 6330 서문시장 저렴한 url',
 '대구서문시장맛집 0507 서문시장 테이블 url',
 '예전푸드몰 검색후 세번째이다 김밥 메뉴',
 '보았습니당 eating 서문시장 월요일 5천',
 '국채보상로99길 맛일까 15분 중국집 url',
 '곳으로 053 10분 할인 url',
 '서문시장칼국수맛집 가격에 먹었어요 서문시장 메뉴',
 '포스팅중이다 블로그 50m 수성호텔 가격',
 '찍어주시고 행복 관광마켓의 운영시간 url',
 '양녕오뎅을 맛집 방문고객들의 대구서문시장 집에',
 '가라한다고해서 9000원 url 50m 생선가스',
 '대구서문시장맛집 김밥 10년째 행복 url',
 '대구전통시장으로 com 5층 url 얼마',
 '협업관계인건가ㅎㅎㅎ url 2023 가격 50m',
 '1년만에 함차wife가 이웃추가

In [16]:
# 키워드 추출 데이터 csv 저장
import csv

# 리스트를 CSV 파일로 저장
with open('strings_Djoined_blog.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    for item in strings_Djoined:
        writer.writerow([item])  # 각 리스트 요소를 한 행에 저장

In [17]:
# 명사, 형용사, 수사, 해시태그, etc 등 분리함수
def okt_token(text):
    okt = Okt()
    words_list = []
    stop_words = ['하다', '있다', '되어다']
    morp = okt.pos(text, stem = True)

    for word, tag in morp:
        if (tag =='Noun' or tag =='Number' or tag == 'Adjective' or tag == 'Hashtag') and (len(word) > 1) and (word not in stop_words):
            words_list.append(word)

    return words_list

## 예산

In [18]:
# 예산시장
Ycv = CountVectorizer(ngram_range = (1, 1),
                     tokenizer = okt_token)

In [19]:
Ydtm_counts = Ycv.fit_transform(strings_Yjoined)

C:\Users\wodnj\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [25]:
# 데이터 확인
Ydtm_df = pd.DataFrame(Ydtm_counts.todense().tolist(),
                     columns = Ycv.get_feature_names_out())
Ydtm_df

,00,0002439633,0007023860,000억,000원,0032,0067,007,01,010,...,휴무,휴업,휴일,흐규규귭,흐름,흡연,흡입,힐링,힘드다,힙합
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1976,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1977,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
Yttm_counts = (Ydtm_counts.T * Ydtm_counts)
# 본인에 해당하는 대각행렬 = 0
Yttm_counts.setdiag(0)

In [27]:
Yttm_df = pd.DataFrame(Yttm_counts.todense().tolist(),
                      columns = Ycv.get_feature_names_out(),
                      index = Ycv.get_feature_names_out())

In [28]:
Yttm_df

,00,0002439633,0007023860,000억,000원,0032,0067,007,01,010,...,휴무,휴업,휴일,흐규규귭,흐름,흡연,흡입,힐링,힘드다,힙합
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002439633,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0007023860,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000억,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000원,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
흡연,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
흡입,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힐링,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힘드다,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# 인접행렬 가져오기
YG = nx.from_pandas_adjacency(Yttm_df)

In [30]:
# Degree of centrality
Ydc = nx.degree_centrality(YG)
Ydc_df = sorted(Ydc.items(), key = lambda x: x[1], reverse = True)
Ydc_df = pd.DataFrame(list(Ydc_df)[:100],
                     columns = ['word', 'dc'])
Ydc_df

,word,dc
0,예산,0.627998
1,시장,0.527166
2,2023,0.235928
3,맥주,0.226628
4,산시,0.219775
...,...,...
95,정보,0.023005
96,9시,0.022516
97,사진,0.022516
98,터미널,0.022516


In [31]:
# closeness of centrality
Ycc = nx.closeness_centrality(YG)
Ycc_df = sorted(Ycc.items(), key = lambda x : x[1], reverse = True)
Ycc_df = pd.DataFrame(list(Ycc_df)[:100],
                     columns = ['word', 'cc'])
Ycc_df

,word,cc
0,예산,0.723810
1,시장,0.673004
2,2023,0.555783
3,산시,0.549742
4,맥주,0.549592
...,...,...
95,15분,0.465176
96,자마자,0.464962
97,3시,0.464855
98,2시,0.464748


In [32]:
# betweenness of centrality
Ybc = nx.betweenness_centrality(YG)
Ybc_df = sorted(Ybc.items(), key = lambda x : x[1], reverse = True)
Ybc_df = pd.DataFrame(list(Ybc_df)[:100],
                     columns = ['word', 'dc'])
Ybc_df

,word,dc
0,예산,0.348794
1,시장,0.208636
2,산시,0.072138
3,2023,0.059371
4,맥주,0.041222
...,...,...
95,우수,0.001722
96,음식,0.001703
97,투어,0.001657
98,사진,0.001637


In [33]:
# centrality 종합표 만들기
Ytemp = pd.merge(Ydc_df, Ycc_df, how = 'outer', on = 'word')
Ycentrality_df = pd.merge(Ytemp, Ybc_df, how = 'outer', on = 'word')
Ycentrality_df

,word,dc_x,cc,dc_y
0,예산,0.627998,0.723810,0.348794
1,시장,0.527166,0.673004,0.208636
2,2023,0.235928,0.555783,0.059371
3,맥주,0.226628,0.549592,0.041222
4,산시,0.219775,0.549742,0.072138
...,...,...,...,...
136,100,NaN,NaN,0.001825
137,아산,NaN,NaN,0.001819
138,추석,NaN,NaN,0.001726
139,우수,NaN,NaN,0.001722


## 대구

In [34]:
cv = CountVectorizer(ngram_range = (1, 1),
                     tokenizer = okt_token)

In [35]:
dtm_counts = cv.fit_transform(strings_Djoined)

C:\Users\wodnj\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
# 데이터 확인
dtm_df = pd.DataFrame(dtm_counts.todense().tolist(),
                     columns = cv.get_feature_names_out())
dtm_df

,00,000,0000045821,000원,0064,010,022,0221,02시,03,...,휴대폰,휴무,휴식,흠뻑,희망,히히히,힐링,힐스,힘들다,힙합
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1820,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1822,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
ttm_counts = (dtm_counts.T * dtm_counts)
# 본인에 해당하는 대각행렬 = 0
ttm_counts.setdiag(0)

In [38]:
ttm_df = pd.DataFrame(ttm_counts.todense().tolist(),
                      columns = cv.get_feature_names_out(),
                      index = cv.get_feature_names_out())

In [39]:
ttm_df

,00,000,0000045821,000원,0064,010,022,0221,02시,03,...,휴대폰,휴무,휴식,흠뻑,희망,히히히,힐링,힐스,힘들다,힙합
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0000045821,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000원,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0064,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
히히히,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힐링,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힐스,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
힘들다,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# 인접행렬 가져오기
G = nx.from_pandas_adjacency(ttm_df)

In [42]:
# Degree of centrality
dc = nx.degree_centrality(G)
dc_df = sorted(dc.items(), key = lambda x: x[1], reverse = True)
dc_df = pd.DataFrame(list(dc_df)[:100],
                     columns = ['word', 'dc'])
dc_df

,word,dc
0,2023,0.313098
1,50,0.219543
2,대구,0.208732
3,서문시장,0.173805
4,페스티벌,0.158836
...,...,...
95,8090,0.021206
96,경북,0.021206
97,대구광역시,0.021206
98,안전,0.021206


In [43]:
# closeness of centrality
cc = nx.closeness_centrality(G)
cc_df = sorted(cc.items(), key = lambda x : x[1], reverse = True)
cc_df = pd.DataFrame(list(cc_df)[:100],
                     columns = ['word', 'cc'])
cc_df

,word,cc
0,2023,0.576422
1,50,0.543187
2,대구,0.534922
3,서문시장,0.518551
4,페스티벌,0.514698
...,...,...
95,요금,0.432714
96,좋다,0.432634
97,야시장,0.431999
98,마켓,0.431920


In [44]:
# betweenness of centrality
bc = nx.betweenness_centrality(G)
bc_df = sorted(bc.items(), key = lambda x : x[1], reverse = True)
bc_df = pd.DataFrame(list(bc_df)[:100],
                     columns = ['word', 'dc'])
bc_df

,word,dc
0,2023,0.217236
1,50,0.127001
2,대구,0.088891
3,서문시장,0.077754
4,맛집,0.045151
...,...,...
95,닭발,0.003334
96,대회,0.003328
97,아프다,0.003288
98,평화시장,0.003220


In [45]:
# centrality 종합표 만들기
temp = pd.merge(dc_df, cc_df, how = 'outer', on = 'word')
centrality_df = pd.merge(temp, bc_df, how = 'outer', on = 'word')
centrality_df

,word,dc_x,cc,dc_y
0,2023,0.313098,0.576422,0.217236
1,50,0.219543,0.543187,0.127001
2,대구,0.208732,0.534922,0.088891
3,서문시장,0.173805,0.518551,0.077754
4,페스티벌,0.158836,0.514698,0.044096
...,...,...,...,...
129,한복,NaN,NaN,0.003505
130,음료,NaN,NaN,0.003461
131,닭발,NaN,NaN,0.003334
132,아프다,NaN,NaN,0.003288


## 기존 코드(무시)

In [ ]:
# 예산 keybert 명사 키워드 추출
ys_festival['noun_keywords'] = ys_festival['keywords'].apply(lambda x : extract_nouns_from_keywords(x))
ys_market['noun_keywords'] = ys_market['keywords'].apply(lambda x : extract_nouns_from_keywords(x))

# 대구 keybert 명사 키워드 추출
dg_festival['noun_keywords'] = dg_festival['keywords'].apply(lambda x : extract_nouns_from_keywords(x))
dg_market['noun_keywords'] = dg_market['keywords'].apply(lambda x : extract_nouns_from_keywords(x))

In [ ]:
# 문자열로 합치기
ys_festival['noun_text'] = ys_festival['noun_keywords'].apply(lambda x: ' '.join(x))
ys_market['noun_text'] = ys_market['noun_keywords'].apply(lambda x: ' '.join(x))
dg_festival['noun_text'] = dg_festival['noun_keywords'].apply(lambda x: ' '.join(x))
dg_market['noun_text'] = dg_market['noun_keywords'].apply(lambda x: ' '.join(x))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), lowercase=False)

# 예산 맥주페스티벌 term to term 매트릭스 생성
X = vectorizer.fit_transform(ys_festival['noun_text'])
co_occurrence_matrix = (X.T * X)
co_occurrence_matrix.setdiag(0)
words = vectorizer.get_feature_names_out()
keywords_ys_festival = pd.DataFrame(data=co_occurrence_matrix.toarray(), index=words, columns=words)

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), lowercase=False)

# 예산 시장 term to term 매트릭스 생성
X = vectorizer.fit_transform(ys_market['noun_text'])
co_occurrence_matrix = (X.T * X)
co_occurrence_matrix.setdiag(0)
words = vectorizer.get_feature_names_out()
keywords_ys_market = pd.DataFrame(data=co_occurrence_matrix.toarray(), index=words, columns=words)

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), lowercase=False)

# 대구치맥페스티벌 term to term 매트릭스 생성
X = vectorizer.fit_transform(dg_festival['noun_text'])
co_occurrence_matrix = (X.T * X)
co_occurrence_matrix.setdiag(0)
words = vectorizer.get_feature_names_out()
keywords_dg_festival = pd.DataFrame(data=co_occurrence_matrix.toarray(), index=words, columns=words)

In [ ]:
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(), lowercase=False)

# 대구 서문시장 term to term 매트릭스 생성
X = vectorizer.fit_transform(dg_market['noun_text'])
co_occurrence_matrix = (X.T * X)
co_occurrence_matrix.setdiag(0)
words = vectorizer.get_feature_names_out()
keywords_dg_market = pd.DataFrame(data=co_occurrence_matrix.toarray(), index=words, columns=words)

In [ ]:
# 4개 검색어 키워드에 대해서 각각 term to term 매트릭스 완성

In [17]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# 공동 출현 행렬로부터 네트워크 그래프 생성
G = nx.from_pandas_adjacency(keywords_ys_festival)

# 네트워크 시각화
plt.figure(figsize=(10, 10))
nx.draw(G, with_labels=True, node_color="skyblue", node_size=2000, edge_color="gray", font_size=12, font_weight='bold')
plt.show()